In [1]:
import os
import sys

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

os.environ['TREEMORT_DATA_PATH'] = '/Users/anisr/Documents/dead_trees'

In [2]:
from treemort.utils.config import setup

config_file_path = "../configs/flair_unet_bs8_cs256.txt"
conf = setup(config_file_path)

# Modified Config Variables for Local Execution; comment on HPC
conf.data_folder = "/Users/anisr/Documents/dead_trees/Finland/RGBNIR/25cm"
conf.output_dir = os.path.join("..", conf.output_dir)
conf.resume = False

print(conf)

Namespace(model='flair_unet', backbone=None, model_weights='best', learning_rate=0.0001, activation='sigmoid', loss='hybrid', threshold=0.5, class_weights=[0.1, 0.9], epochs=100, train_batch_size=8, val_batch_size=8, test_batch_size=8, train_crop_size=256, val_crop_size=256, test_crop_size=256, val_size=0.2, test_size=0.1, resume=False, data_folder='/Users/anisr/Documents/dead_trees/Finland/RGBNIR/25cm', hdf5_file='Finland_RGBNIR_25cm_sdt.h5', input_channels=4, output_channels=3, output_dir='.././output/flair_unet_sdt')


In [3]:
from treemort.main import run

eval_only = False

run(conf, eval_only)

/Users/anisr/Documents/TreeSeg/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Training:  50%|█████     | 10/20 [00:30<00:30,  3.09s/batch, Loss=1.6953, IOU=0.0041, F1=0.0081]


KeyboardInterrupt: 